In [1]:
import sys, os
import importlib.metadata

# Ajusta si tu carpeta local no se llama exactamente bullet3-3.25
sys.path.append(os.path.abspath("bullet3-3.25"))

import pybullet as p
import pybullet_data
import gymnasium as gym
import numpy as np
import torch

print("✅ PyBullet", importlib.metadata.version("pybullet"))
print("✅ Gymnasium", gym.__version__)
print("✅ Torch", torch.__version__)


✅ PyBullet 3.2.7
✅ Gymnasium 1.1.1
✅ Torch 2.5.1+cu121


In [2]:
from gymnasium import spaces
import numpy as np
import pybullet as p
import pybullet_data

class RobotCupEnv(gym.Env):
    """Brazo Panda que voltea un vaso cilíndrico, con reward rebalanceado."""
    metadata = {"render_modes": ["human", "direct"], "render_fps": 240}

    def __init__(self, render_mode="direct"):
        super().__init__()
        self.render_mode = render_mode
        self.client = p.connect(p.GUI if render_mode=="human" else p.DIRECT)

        # Datos de PyBullet
        p.setAdditionalSearchPath(pybullet_data.getDataPath())
        self.time_step = 1/240
        p.setTimeStep(self.time_step, physicsClientId=self.client)
        p.setGravity(0,0,-9.8, physicsClientId=self.client)

        # Espacio de acciones (7 ángulos)
        self.action_space = spaces.Box(
            low  = np.array([-2.9]*7, dtype=np.float32),
            high = np.array([ 2.9]*7, dtype=np.float32)
        )
        # Observación: 7 ángulos + (x,y,z) del vaso
        low  = np.array([-3.2]*7 + [-1.0, -1.0, 0.0], dtype=np.float32)
        high = np.array([ 3.2]*7 + [ 1.0,  1.0, 1.0], dtype=np.float32)
        self.observation_space = spaces.Box(low, high, dtype=np.float32)

        self.max_steps    = 240 * 5
        self.step_counter = 0

    def reset(self, *, seed=None, options=None):
        # Reiniciar simulación
        p.resetSimulation(physicsClientId=self.client)
        p.setGravity(0,0,-9.8, physicsClientId=self.client)
        p.setTimeStep(self.time_step, physicsClientId=self.client)
        p.loadURDF("plane.urdf", physicsClientId=self.client)

        # Cargar robot
        self.robot_id = p.loadURDF(
            "franka_panda/panda.urdf",
            basePosition=[0,0,0],
            useFixedBase=True,
            physicsClientId=self.client
        )
        # Perturbar postura inicial ±5°
        for j in range(7):
            delta = self.np_random.uniform(-0.087, 0.087)
            p.resetJointState(self.robot_id, j, delta, physicsClientId=self.client)

        # Crear vaso en X∈[-0.3,0.3], Y∈[-0.1,0.1]
        col = p.createCollisionShape(p.GEOM_CYLINDER, radius=0.03, height=0.1,
                                     physicsClientId=self.client)
        vis = p.createVisualShape(p.GEOM_CYLINDER, radius=0.03, length=0.1,
                                  rgbaColor=[1,1,1,1],
                                  physicsClientId=self.client)
        x = self.np_random.uniform(-0.3, 0.3)
        y = self.np_random.uniform(-0.1, 0.1)
        self.cup_id = p.createMultiBody(
            baseMass=0.1,
            baseCollisionShapeIndex=col,
            baseVisualShapeIndex=vis,
            basePosition=[x, y, 0.05],
            physicsClientId=self.client
        )

        self.step_counter = 0
        return self._get_obs(), {}

    def step(self, action):
        # Aplicar acción de posición
        for i, tgt in enumerate(action):
            p.setJointMotorControl2(
                self.robot_id, jointIndex=i,
                controlMode=p.POSITION_CONTROL,
                targetPosition=tgt,
                physicsClientId=self.client
            )
        p.stepSimulation(physicsClientId=self.client)
        self.step_counter += 1

        # Calcular tilt actual
        _, quat = p.getBasePositionAndOrientation(self.cup_id,
                                                  physicsClientId=self.client)
        roll, pitch, _ = p.getEulerFromQuaternion(quat)
        tilt = max(abs(roll), abs(pitch))

        obs    = self._get_obs()
        reward = self._compute_reward(tilt)
        done   = self.step_counter >= self.max_steps
        # Termina tan pronto alcanza 0.8 rad (~45°)
        terminated = done or (tilt >= 0.8)

        return obs, reward, terminated, False, {}

    def _get_obs(self):
        # Ángulos de juntas
        js = p.getJointStates(self.robot_id, list(range(7)),
                              physicsClientId=self.client)
        angles = [s[0] for s in js]
        # Posición del vaso
        pos, _ = p.getBasePositionAndOrientation(self.cup_id,
                                                  physicsClientId=self.client)
        return np.array(angles + list(pos), dtype=np.float32)

    def _compute_reward(self, tilt: float) -> float:
        # tilt ∈ [0, π/2]
        r_tilt = np.clip(tilt / (np.pi/2), 0, 1)
        # bonus extra si pasa 45° (0.785 rad)
        bonus = 0.5 if tilt > 0.785 else 0.0
        # penalización suave
        penalty = 0.001
        return float(r_tilt + bonus - penalty)

    def render(self):
        pass

    def close(self):
        p.disconnect(physicsClientId=self.client)


In [3]:
env = RobotCupEnv(render_mode="direct")
obs, _ = env.reset()
for _ in range(200):
    a = env.action_space.sample()
    obs, r, done, trunc, _ = env.step(a)
    if done or trunc:
        break
env.close()
print("✅ Test direct completado | Reward:", r)


✅ Test direct completado | Reward: -0.0008931550809759968


In [4]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecMonitor
from stable_baselines3.common.callbacks import EvalCallback

# — training env —
venv = DummyVecEnv([lambda: RobotCupEnv(render_mode="direct") for _ in range(4)])
venv = VecMonitor(venv)

# — eval env del mismo tipo —
eval_venv = DummyVecEnv([lambda: RobotCupEnv(render_mode="direct")])
eval_venv = VecMonitor(eval_venv)

# — callback de evaluación —
eval_cb = EvalCallback(
    eval_venv,
    best_model_save_path="./logs/",
    log_path="./logs/",
    eval_freq=20_000,
    n_eval_episodes=5,
    deterministic=True,
    render=False,
)

model = PPO(
    "MlpPolicy",
    venv,
    verbose=1,
    device="cpu",
    n_steps=256,
    learning_rate=1e-4,
    ent_coef=0.05,
    batch_size=64,
)

model.learn(total_timesteps=500_000, callback=eval_cb)
model.save("ppo_robot_cup_no_norm")


Using cpu device
-----------------------------
| time/              |      |
|    fps             | 537  |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 1024 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 498          |
|    iterations           | 2            |
|    time_elapsed         | 4            |
|    total_timesteps      | 2048         |
| train/                  |              |
|    approx_kl            | 0.0078258095 |
|    clip_fraction        | 0.0304       |
|    clip_range           | 0.2          |
|    entropy_loss         | -9.94        |
|    explained_variance   | -0.81        |
|    learning_rate        | 0.0001       |
|    loss                 | -0.521       |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.0107      |
|    std                  | 1            |
|    value_loss           | 0.00621      |

In [6]:
from stable_baselines3 import PPO

model = PPO.load("ppo_robot_cup_no_norm", device="cpu")
env   = RobotCupEnv(render_mode="human")

obs, _       = env.reset()
total_reward = 0.0

for _ in range(500):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, trunc, _ = env.step(action)
    total_reward += reward
    if done or trunc:
        break

# Tilt final
pos, quat = p.getBasePositionAndOrientation(env.cup_id, physicsClientId=env.client)
roll, pitch, _ = p.getEulerFromQuaternion(quat)
env.close()

print(f"🎯 Recompensa acumulada: {total_reward:.3f}")
print(f"🔄 Tilt final → roll: {roll:.2f} rad, pitch: {pitch:.2f} rad")


🎯 Recompensa acumulada: 6.150
🔄 Tilt final → roll: -0.84 rad, pitch: 0.11 rad


In [ ]:
import numpy as np
from stable_baselines3 import PPO

# 1) (Re)crea el env HUMAN para evaluación
env   = RobotCupEnv(render_mode="human")
model = PPO.load("ppo_robot_cup_no_norm", device="cpu")

n_eval = 20
rewards, lengths, tilts = [], [], []

for _ in range(n_eval):
    obs, _     = env.reset()
    total_r, t = 0.0, 0
    while True:
        action, _ = model.predict(obs, deterministic=True)
        obs, r, done, trunc, _ = env.step(action)
        total_r += r
        t       += 1
        if done or trunc:
            break
    # tilt final
    _, quat = p.getBasePositionAndOrientation(env.cup_id, physicsClientId=env.client)
    roll, _, _ = p.getEulerFromQuaternion(quat)

    rewards.append(total_r)
    lengths.append(t)
    tilts.append(abs(roll))

env.close()

print(f"Rew mean/std: {np.mean(rewards):.2f} ± {np.std(rewards):.2f}")
print(f"Steps mean/std: {np.mean(lengths):.1f} ± {np.std(lengths):.1f}")
print(f"Tilt mean/std: {np.mean(tilts):.2f} rad ± {np.std(tilts):.2f}")



Rew mean/std: 1.39 ± 4.36
Steps mean/std: 847.8 ± 538.1
Tilt mean/std: 0.17 rad ± 0.33
